In [45]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.metrics import classification_report
import torch

# Load dataset
file_path = '/content/tweet_financial_validation_set_final.csv'
df = pd.read_csv(file_path)

# Ensure labels are in the correct format
label_mapping = {1: 2, 2: 0, 0: 1}  # Negative: 0, Neutral: 1, Positive: 2
df['label'] = df['label'].map(label_mapping)

# Check for NaN values in the 'label' column after mapping
if df['label'].isnull().any():
    print("There are NaN values in the label column after mapping.")
else:
    print("No NaN values in the label column after mapping.")

# Check for and drop any rows with missing values
df.dropna(subset=['text', 'label'], inplace=True)

texts = df['text'].tolist()
labels = df['label'].tolist()

# Split the data
train_texts, val_texts, train_labels, val_labels = train_test_split(texts, labels, test_size=0.1, stratify=labels)


No NaN values in the label column after mapping.


In [40]:
def evaluate_model_with_report(model_name, texts, labels, batch_size=32):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSequenceClassification.from_pretrained(model_name)
    model.eval()

    all_predictions = []

    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i+batch_size]
        inputs = tokenizer(batch_texts, return_tensors='pt', truncation=True, padding=True, max_length=512)
        with torch.no_grad():
            outputs = model(**inputs)
        predictions = torch.argmax(outputs.logits, dim=-1).numpy()
        all_predictions.extend(predictions)

    report = classification_report(labels, all_predictions, target_names=['negative', 'neutral', 'positive'], output_dict=True)
    return report, all_predictions


In [41]:
def evaluate_gpt2_model_with_report(texts, labels, batch_size=32):
    model_name = "gpt2"
    tokenizer = GPT2Tokenizer.from_pretrained(model_name)
    model = GPT2ForSequenceClassification.from_pretrained(model_name)

    # Add padding token for GPT-2
    tokenizer.pad_token = tokenizer.eos_token
    model.config.pad_token_id = tokenizer.eos_token_id
    model.eval()

    all_predictions = []

    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i+batch_size]
        inputs = tokenizer(batch_texts, return_tensors='pt', truncation=True, padding=True, max_length=512)
        with torch.no_grad():
            outputs = model(**inputs)
        predictions = torch.argmax(outputs.logits, dim=-1).numpy()
        all_predictions.extend(predictions)

    report = classification_report(labels, all_predictions, target_names=['negative', 'neutral', 'positive'], output_dict=True)
    return report, all_predictions


In [42]:
# FinBERT model
finbert_model = "yiyanghkust/finbert-tone"
finbert_report, finbert_predictions = evaluate_model_with_report(finbert_model, val_texts, val_labels)
print("FinBERT Classification Report:")
print(classification_report(val_labels, finbert_predictions, target_names=['negative', 'neutral', 'positive']))


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


FinBERT Classification Report:
              precision    recall  f1-score   support

    negative       0.71      0.79      0.75       157
     neutral       0.03      0.03      0.03        35
    positive       0.10      0.06      0.08        47

    accuracy                           0.54       239
   macro avg       0.28      0.29      0.29       239
weighted avg       0.49      0.54      0.51       239



In [44]:
# GPT-2 model
gpt2_report, gpt2_predictions = evaluate_gpt2_model_with_report(val_texts, val_labels)
print("GPT-2 Classification Report:")
print(classification_report(val_labels, gpt2_predictions, target_names=['negative', 'neutral', 'positive']))


Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


GPT-2 Classification Report:
              precision    recall  f1-score   support

    negative       0.66      1.00      0.79       157
     neutral       0.00      0.00      0.00        35
    positive       0.00      0.00      0.00        47

    accuracy                           0.66       239
   macro avg       0.22      0.33      0.26       239
weighted avg       0.43      0.66      0.52       239



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classif

In [29]:
# BERT model
bert_model = "bert-base-uncased"
bert_report, bert_predictions = evaluate_model_with_report(bert_model, val_texts, val_labels)
print("BERT Classification Report:")
print(classification_report(val_labels, bert_predictions, target_names=['negative', 'neutral', 'positive']))

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BERT Classification Report:
              precision    recall  f1-score   support

    negative       0.67      0.83      0.74       157
     neutral       0.20      0.26      0.22        35
    positive       0.00      0.00      0.00        47

    accuracy                           0.58       239
   macro avg       0.29      0.36      0.32       239
weighted avg       0.47      0.58      0.52       239



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classif

In [30]:
#RoBERTa model
roberta_model = "roberta-base"
roberta_report, roberta_predictions = evaluate_model_with_report(roberta_model, val_texts, val_labels)
print("RoBERTa Classification Report:")
print(classification_report(val_labels, roberta_predictions, target_names=['neutral', 'negative', 'positive']))


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RoBERTa Classification Report:
              precision    recall  f1-score   support

     neutral       0.00      0.00      0.00       157
    negative       0.15      1.00      0.26        35
    positive       0.00      0.00      0.00        47

    accuracy                           0.15       239
   macro avg       0.05      0.33      0.09       239
weighted avg       0.02      0.15      0.04       239



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classif